In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import sys
from typing import Dict, List, Optional, Tuple, Union
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec19.txt'

In [2]:
ops = sim.read_ops(INPUT_PATH.read_text().strip())

In [3]:
class OutputWrapper:
    def __init__(self):
        self.painted = {}
        self.pos = None
    
    def set_pos(self, pos_x, pos_y):
        self.pos = (pos_x, pos_y)

    def __call__(self, val):
        self.painted[self.pos] = val

def paint(x_range, y_range):
    output = OutputWrapper()
    for y in range(y_range):
        for x in range(x_range):
            output.set_pos(x, y)
            sim.simulate(ops, inputs=[x, y], output_func=output)
    return output.painted

def print_beam(painted):
    min_x = min(key[0] for key in painted.keys())
    max_x = max(key[0] for key in painted.keys())
    min_y = min(key[1] for key in painted.keys())
    max_y = max(key[1] for key in painted.keys())
    
    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            if painted[(x, y)]:
                print('#', end='')
            else:
                print('.', end='')
        print()

In [4]:
painted = paint(50, 50)
print(f'The answer to part 1 is {sum(painted.values())}')

The answer to part 1 is 199


In [5]:
print_beam(painted)

#.................................................
..................................................
...#..............................................
....#.............................................
.....##...........................................
.......#..........................................
........##........................................
.........##.......................................
..........###.....................................
............##....................................
.............###..................................
..............###.................................
...............####...............................
.................####.............................
..................####............................
...................#####..........................
....................#####.........................
.....................######.......................
.......................#####......................
........................######.

In [6]:
# Get some sort of midrange slope
painted = paint(100, 100)

In [7]:
min_ys = {y: min(x for x in range(100) if painted[(x, y)]) for y in range(10, 80)}
max_ys = {y: max(x for x in range(100) if painted[(x, y)]) for y in range(10, 80) if not painted[(99, y)]}

In [8]:
min_slope = sum(x / y for y, x in min_ys.items()) / len(min_ys)

In [9]:
min_slope

1.2490179997146647

In [10]:
max_slope = sum(x / y for y, x in max_ys.items()) / len(max_ys)

In [11]:
max_slope

1.5224051574027693

In [12]:
# So we need to find x and y s.t.
#  & m * y < x
#  & M * y > x + 99
#  & m * (y + 99) < x and
#  & M * (y + 99) > x + 99
#
# my < x - 99m
# My > x + 99
#
# So x = my + 99m
#    x = My - 99
# my + 99m = My - 99
# y = 99(m + 1) / (M - m)


In [13]:
w = 99
start_y = w * (min_slope + 1) / (max_slope - min_slope)

assert abs((max_slope * start_y - w) - (min_slope * start_y + w * min_slope)) < 1e-7
start_x = max_slope * start_y - w

In [14]:
print(start_x, start_y)

start_x = int(start_x)
start_y = int(start_y)

1140.8817356687348 814.4229738317356


In [15]:
class Capture:
    def __init__(self):
        self.output = None

    def __call__(self, val):
        self.output = val

def does_work(start_x, start_y):
    # Checks if a 100 x 100 square works
    for x, y in [(start_x, start_y), (start_x + 99, start_y), (start_x, start_y + 99), (start_x + 99, start_y + 99)]:
        capture = Capture()
        sim.simulate(ops, inputs=[x, y], output_func=capture)
        if capture.output != 1:
            return False
    return True

assert does_work(start_x, start_y)

In [16]:
# Now get exact numbers
xx, yy = start_x, start_y
while True:
    new_all_work = []
    for x in range(xx - 20, xx + 1):
        for y in range(yy - 20, yy + 1):
            if x == xx and y == yy:
                continue
            if does_work(x, y):
                new_all_work.append((x, y))
    if new_all_work:
        xx, yy = min(new_all_work, key=lambda x: x[0] + x[1])
    else:
        break


In [17]:
print(f'The answer to part 2 is {10000 * xx + yy}')

The answer to part 2 is 10180726
